In [1]:
import os
import shutil
from tqdm import tqdm
import csv
import random
import matplotlib.pyplot as plt
import numpy as np
import time

In [14]:
datapath = 'Dataset-5'
extra = 'extras'
train_path = 'data_train'
test_path = 'data_test'
validation_path = 'data_validation'

In [15]:
# speakers, n = np.load('details.npy',allow_pickle=True)

In [16]:
def get_labels(path):
    labels = os.listdir(path)
    return labels, len(labels)
speakers, n = get_labels(datapath)

# Saving the names of speakers and their labels
details=[speakers,n]
np.save('details.npy', details)
print('Number of speakers: ',n)

Number of speakers:  5


In [17]:
speakers[:5]

['id10003', 'id10004', 'id10005', 'id10006', 'id10007']

In [20]:
data_sources = [train_path,test_path,validation_path,extra]

In [21]:
for source in data_sources:
    try:
        # Create target Directory
        os.mkdir(source)
        print("Directory " , source ,  " Created ") 
    except FileExistsError:
        print("Directory " , source ,  " already exists")

Directory  data_train  already exists
Directory  data_test  already exists
Directory  data_validation  already exists
Directory  extras  already exists


In [22]:
# if exists then clearing it
for source in data_sources:
    fileList = os.listdir(source)
    for fileName in fileList:
        os.remove(source+"/"+fileName)

##### Making Train and Test csv files containing the audio filename and the speaker associated with that audio

In [23]:
def organise_data_csv(path):
    validation = [['SampleName','Speaker']]
    test = [['SampleName','Speaker']]
    train =[['SampleName','Speaker']]
    training_samples = [[] for _ in range(n)]
    for id_ in speakers:
        c = 0
        val = []
        test_id = []
        train_id = []
        remaining = []
        speaker_path = path+'/'+id_
        videos = os.listdir(speaker_path)
        
        random.shuffle(videos)
        for v in videos:
            files = os.listdir(speaker_path+'/'+v) 
            random.shuffle(files)
            if len(files) >= 3:
                if len(train_id) < 15:
                    c +=1
                    audio_clip = speaker_path+'/'+v+'/'+files.pop(0), train_path +'/'+id_+'.'+str(c)+'.wav'
                    shutil.move(audio_clip[0],audio_clip[1])
                    train_id.append([audio_clip[1],id_])
                if len(val) < 5:
                    c +=1
                    audio_clip = speaker_path+'/'+v+'/'+files.pop(0), validation_path +'/'+id_+'.'+str(c)+'.wav'
                    shutil.move(audio_clip[0],audio_clip[1])
                    val.append([audio_clip[1],id_])
                if len(test_id) < 5:
                    c +=1
                    audio_clip = speaker_path+'/'+v+'/'+files.pop(0), test_path +'/'+id_+'.'+str(c)+'.wav'
                    shutil.move(audio_clip[0],audio_clip[1])
                    test_id.append([audio_clip[1],id_])
            else:
                if len(train_id) < 15:
                    c +=1
                    audio_clip = speaker_path+'/'+v+'/'+files.pop(0), train_path +'/'+id_+'.'+str(c)+'.wav'
                    shutil.move(audio_clip[0],audio_clip[1])
                    train_id.append([audio_clip[1],id_])
            while (len(files) != 0):
                c +=1
                audio_clip = speaker_path+'/'+v+'/'+files.pop(0), extra +'/'+id_+'.'+str(c)+'.wav'
                shutil.move(audio_clip[0],audio_clip[1])
                remaining.append(audio_clip[1])
            shutil.rmtree(speaker_path+'/'+v)
                
        random.shuffle(remaining)
        while (len(train_id) < 15):
            if len(remaining) > 0:
                c+=1
                audio_clip = remaining.pop(0), train_path +'/'+id_+'.'+str(c)+'.wav'
                shutil.move(audio_clip[0],audio_clip[1])
                train_id.append([audio_clip[1],id_])
        while (len(test_id) < 5):
            if len(remaining) > 0:
                c+=1
                audio_clip = remaining.pop(0), test_path +'/'+id_+'.'+str(c)+'.wav'
                shutil.move(audio_clip[0],audio_clip[1])
                test_id.append([audio_clip[1],id_])
        while (len(val) < 5):
            if len(remaining) > 0:
                c+=1
                audio_clip = remaining.pop(0), validation_path +'/'+id_+'.'+str(c)+'.wav'
                shutil.move(audio_clip[0],audio_clip[1])
                val.append([audio_clip[1],id_])
                
            
        validation.extend(val)
        test.extend(test_id)
        training_samples[speakers.index(id_)].extend(train_id)
        train.extend(train_id)
        shutil.rmtree(path+'/'+id_)
        fileList = os.listdir(extra)
        for fileName in fileList:
             os.remove(extra+"/"+fileName)
        
        
    with open('train.csv', 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(train)
    csvFile.close()    
    
    with open('test.csv', 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(test)
    csvFile.close()
    
    with open('validation.csv', 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(validation)
    csvFile.close() 
    
    return training_samples,test, validation
start = time.time()      
training_samples ,testing_samples, validation_samples = organise_data_csv(datapath)
end = time.time()
print('Time for',n,'speakers', (end-start)/60,'mins')

Time for 5 speakers 0.015128906567891438 mins


In [24]:
np.save('training_samples.npy', training_samples)
np.save('testing_samples.npy', testing_samples)
np.save('validation_samples.npy', validation_samples)

In [25]:
# training_samples = np.load('training_samples.npy', allow_pickle=True).tolist()
# testing_samples = np.load('testing_samples.npy', allow_pickle=True).tolist()
# validation_samples = np.load('validatiom_samples.npy', allow_pickle=True).tolist()

In [26]:
sum([len(i) for i in training_samples]) == 15*n

True